# 01-数据加载模块

下面展示数据加载模块的效果，这里以下载 NVDA 股票的数据为例

In [2]:
import os
import sys

# 将项目根目录添加到 python 路径中
project_root = os.path.dirname(os.getcwd())
if project_root not in sys.path:
    sys.path.insert(0, project_root)

from scripts.data_loader import DataLoader

loader = DataLoader()
# 抓取 NVIDIA 的数据作为练习
df = loader.fetch_and_save("NVDA", "2023-01-01", "2024-01-01")

开始同步 NVDA 数据 （2023-01-01 至 2024-01-01）...


[*********************100%***********************]  1 of 1 completed

数据成功保存至：../storage/raw\NVDA_20230101_20240101.csv


下面展示的是我们获取数据的效果

In [3]:
# 检查数据质量
df

Price,Close,High,Low,Open,Volume
Ticker,NVDA,NVDA,NVDA,NVDA,NVDA
Date,,,,,
2023-01-03,14.300683,14.981003,14.081902,14.836147,401277000
2023-01-04,14.734251,14.838147,14.226760,14.552434,431324000
2023-01-05,14.250733,14.549434,14.133850,14.476507,389168000
2023-01-06,14.844140,14.994989,14.019965,14.459525,405044000
2023-01-09,15.612372,16.039944,15.125859,15.268717,504231000
...,...,...,...,...,...
2023-12-22,48.802811,49.355500,48.440010,49.167605,252507000
2023-12-26,49.251556,49.572377,48.932734,48.940729,244200000


下面，我们来检查一下下载的数据

In [4]:
# 检查是否有缺失交易日
print(f"缺失值统计:\n{df.isnull().sum()}")

# 检查数据重复
print(f"重复行数: {df.duplicated().sum()}")

缺失值统计:
Price   Ticker
Close   NVDA      0
High    NVDA      0
Low     NVDA      0
Open    NVDA      0
Volume  NVDA      0
dtype: int64
重复行数: 0
